In [1]:
#RoBERTa is BERT variation but following changes
''' -Use dynamic mask, instead of static mask
     -Remove the NSP task and train using the MLM task
     -Train with latge batch size
     -use byte-level BPE(BBPE) as a tokenizer
     '''

' -Use dynamic mask, instead of static mask\n     -Remove the NSP task and train using the MLM task\n     -Train with latge batch size\n     -use byte-level BPE(BBPE) as a tokenizer\n     '

In [2]:
#SOP basically a classification task, classify pair of words belongs to positive or negative class(Order of word not swapped)
 #or negative class(order of word swapped)


In [3]:
from transformers import RobertaConfig, RobertaModel, RobertaModel, RobertaTokenizer
import pandas as pd
import numpy as np
import nltk
from sklearn.metrics.pairwise import cosine_similarity
import torch
from sentence_transformers import SentenceTransformer, util

In [4]:
roberta=RobertaModel.from_pretrained('roberta-base',output_hidden_states=True)
roberta.config

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

In [5]:
tokenizer=RobertaTokenizer.from_pretrained('roberta-base')

In [6]:
path=r"C:\Users\uttam\OneDrive\Desktop\NLP Project\NLP-Project\NLP-Dataset.csv"
df=pd.read_csv(path,sep=None,engine='python')

In [7]:
train_data=df['sentence'].loc[0:2]
test_data=df['sentence'].loc[3:]
train_data

0    My name is uttam asodariya
1        I am ambitious student
2             I like travelling
Name: sentence, dtype: object

In [12]:
# Cosine similarity
model = SentenceTransformer('roberta-base')
embedding=model.encode(train_data)
embedding.shape
cosine_scores = util.cos_sim(embedding, embedding)
cosine_scores

Downloading:   0%|          | 0.00/445 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/603k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

No sentence-transformers model found with name C:\Users\uttam/.cache\torch\sentence_transformers\roberta-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at C:\Users\uttam/.cache\torch\sentence_transformers\roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[1.0000, 0.9542, 0.9307],
        [0.9542, 1.0000, 0.9757],
        [0.9307, 0.9757, 1.0000]])

In [10]:
def Word_embedding(dataframe):
    encoded_tensor=[]
    for i in dataframe:
        input = torch.tensor(tokenizer.encode(i, add_special_tokens=True)).unsqueeze(0)
        
        
        with torch.no_grad():
            output = roberta(input)
            last_hidden_states=output[0]
         
        last_hidden_states=np.array(last_hidden_states)
        sum_of_encoders=np.sum(last_hidden_states,axis=0)
        tensor=sum_of_encoders/12 
# we have 12 encoder output(length of 768) per token. Therefor average it
        encoded_tensor.append(tensor)
    return encoded_tensor

         
encoded_tensor=Word_embedding(train_data)      
encoded_tensor[0]

array([[-0.00380495,  0.0066697 , -0.0023425 , ..., -0.00875238,
        -0.00319631, -0.00455428],
       [-0.00872644,  0.00553709,  0.00703505, ..., -0.00305844,
        -0.00150916, -0.01476572],
       [-0.00223564,  0.006395  ,  0.00709376, ..., -0.02263761,
        -0.001949  , -0.01004509],
       ...,
       [-0.00790431,  0.00265047,  0.01174918, ..., -0.03812897,
        -0.01115266, -0.0071437 ],
       [ 0.00882291,  0.00030284,  0.00789095, ..., -0.02823584,
        -0.0073701 , -0.01307125],
       [-0.00331656,  0.00618941, -0.00465911, ..., -0.01265754,
        -0.00314743, -0.00760065]], dtype=float32)

In [181]:
def cosine(encoded_tensor): 
    encoded_tensor1=encoded_tensor[0]
    encoded_tensor2=encoded_tensor[1]
    
    print('cosine similarity between sentence 1&2', cosine_similarity(encoded_tensor1,
                                                                        encoded_tensor2))


cosine similarity between sentence 1&2 [[0.9987668  0.8235894  0.80704814 0.7604036  0.80606145 0.99452466]
 [0.82093227 0.93390083 0.8927067  0.8290058  0.8638848  0.82419807]
 [0.6305332  0.80441976 0.84412813 0.7978529  0.8049452  0.6363224 ]
 [0.68474334 0.8603505  0.9089815  0.8269222  0.83296317 0.690841  ]
 [0.72782403 0.77462906 0.8077069  0.7993326  0.8073348  0.73623234]
 [0.71977025 0.7814284  0.8028483  0.7847976  0.7868681  0.73200077]
 [0.75327784 0.84998053 0.87524927 0.8394208  0.8666037  0.764046  ]
 [0.6799656  0.750145   0.8050009  0.77797407 0.7694006  0.6898856 ]
 [0.6359     0.7449647  0.7940486  0.7899357  0.7950175  0.64920956]
 [0.7375224  0.82850647 0.8224846  0.81816655 0.84501475 0.7474714 ]
 [0.7651556  0.86588365 0.8757618  0.8586716  0.895524   0.7756463 ]
 [0.9959017  0.82889813 0.8149374  0.767716   0.8180362  0.99801546]]
